## input embeddings

In [48]:
import math
import numpy as np
import torch
L, dk, dv = 4,8,8

q = torch.rand(L, dk)
k = torch.rand(L, dk)
v = torch.rand(L, dv)

attention = (q @ k.T )/ math.sqrt(dk)
attention

tensor([[0.4538, 0.7531, 0.3474, 0.6928],
        [0.4008, 0.5153, 0.2191, 0.4421],
        [0.7622, 1.2977, 0.4161, 0.8186],
        [0.6235, 0.9334, 0.4064, 0.7614]])

In [49]:
q.var(),k.var(), attention.var()

(tensor(0.0776), tensor(0.0812), tensor(0.0734))

In [50]:
tri = torch.tril(torch.ones(4,4))
tri[tri == 0] = -torch.inf
tri[tri == 1] = 0
scaled = attention + tri

In [52]:
torch.softmax(scaled, dim = -1)

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4714, 0.5286, 0.0000, 0.0000],
        [0.2928, 0.5001, 0.2071, 0.0000],
        [0.2317, 0.3159, 0.1865, 0.2660]])

In [55]:
attention.shape, v.shape

(torch.Size([4, 4]), torch.Size([4, 8]))

In [73]:
k = torch.rand(4,8)
k.shape[-1]

8

In [77]:
def self_attention(k, q, v, mask=None):
    
    attention = (k @q.T) / math.sqrt(k.shape[-1])
    if mask is not None:
        attention = attention + mask
    
    scaled_attention = softmax(attention)
    out = scaled_attention @ v
    return out, attention,scaled_attention

def softmax(attention):
    return torch.softmax(attention,dim=-1)

In [80]:
out, att,sc = self_attention(k,q,v,mask=tri)

out, att, sc, v

(tensor([[0.2735, 0.9688, 0.0151, 0.2840, 0.5233, 0.7624, 0.9583, 0.4519],
         [0.1744, 0.7238, 0.1919, 0.3430, 0.4853, 0.5910, 0.7559, 0.6465],
         [0.5019, 0.8354, 0.5186, 0.6100, 0.4572, 0.4551, 0.8194, 0.5343],
         [0.3700, 0.7754, 0.5910, 0.4546, 0.3627, 0.4088, 0.7136, 0.5396]]),
 tensor([[0.4418,   -inf,   -inf,   -inf],
         [0.6809, 0.4334,   -inf,   -inf],
         [0.8124, 0.4710, 0.9954,   -inf],
         [0.3359, 0.3480, 0.5736, 0.5095]]),
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5615, 0.4385, 0.0000, 0.0000],
         [0.3435, 0.2441, 0.4124, 0.0000],
         [0.2237, 0.2264, 0.2837, 0.2661]]),
 tensor([[0.2735, 0.9688, 0.0151, 0.2840, 0.5233, 0.7624, 0.9583, 0.4519],
         [0.0474, 0.4100, 0.4184, 0.4186, 0.4365, 0.3715, 0.4966, 0.8958],
         [0.9611, 0.9760, 0.9973, 0.9948, 0.4144, 0.2487, 0.8947, 0.3891],
         [0.0953, 0.7097, 0.7888, 0.0529, 0.1096, 0.3139, 0.4995, 0.4708]]))

In [89]:
import torch
import torch.nn as nn

seq_len = 4
batch_size =1 
input_dim = 512
d_model = 512

x = torch.rand((batch_size, seq_len , input_dim))

In [104]:
x

tensor([[[0.5269, 0.1897, 0.8433,  ..., 0.6816, 0.6451, 0.4034],
         [0.0545, 0.9215, 0.4575,  ..., 0.4892, 0.6535, 0.8174],
         [0.7029, 0.7112, 0.7370,  ..., 0.4391, 0.2639, 0.1980],
         [0.7231, 0.9426, 0.8046,  ..., 0.4923, 0.3170, 0.9782]]])

In [117]:
qkv_layer = nn.Linear(input_dim,d_model*3)
qkv = qkv_layer(x)

qkv.shape

torch.Size([1, 4, 1536])

In [123]:
num_heads = 8
head_dim = d_model // num_heads

qkv = qkv.reshape(batch_size,seq_len,num_heads,head_dim*3)

In [124]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [125]:
qkv = qkv.permute(0,2,1,3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [126]:
q,k,v = qkv.chunk(3,dim=-1)


In [127]:
q.shape,v.shape,k.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

In [134]:
q.transpose(-1,-2).shape

torch.Size([1, 8, 64, 4])

In [136]:
a= torch.matmul(q, k.transpose(-1,-2))
a.shape

torch.Size([1, 8, 4, 4])

In [137]:
dk = k.shape[-1]
scaled = torch.softmax(torch.matmul(q, k.transpose(-1,-2))/math.sqrt(dk), dim=-1)

In [138]:
scaled.shape

torch.Size([1, 8, 4, 4])

In [157]:
mask = torch.full(scaled.shape,float('-inf'))
mask[0,0,:,:]

tensor([[-inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf]])

In [159]:
mask = torch.triu(mask, diagonal=1)
mask[0,0,:,:]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [160]:
mask += scaled
mask[:,0,:,:]

tensor([[[0.2589,   -inf,   -inf,   -inf],
         [0.2492, 0.2463,   -inf,   -inf],
         [0.2507, 0.2354, 0.2499,   -inf],
         [0.2544, 0.2305, 0.2565, 0.2586]]], grad_fn=<SliceBackward0>)

In [161]:
sof = torch.softmax(mask,dim=-1)

In [162]:
sof[0,0,:,:]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5007, 0.4993, 0.0000, 0.0000],
        [0.3351, 0.3300, 0.3349, 0.0000],
        [0.2511, 0.2452, 0.2516, 0.2521]], grad_fn=<SliceBackward0>)

In [163]:
def scaled_dot(q,k,v,mask=None):
    dk = k.shape[-1]
    attention = torch.matmul(q,k.transpose(-1,-2))/ math.sqrt(dk)
    if mask is not None:
        attention = attention +mask
    scaled = torch.softmax(attention, dim=-1)
    val = scaled @ v
    return scaled, val
    

In [164]:
a, b = scaled_dot(q,k,v)
a.shape, b.shape

(torch.Size([1, 8, 4, 4]), torch.Size([1, 8, 4, 64]))

In [2]:
import torch
import torch.nn as nn

max_seq_len = 10
d_model = 6

In [3]:
even_i = torch.arange(0, d_model,2).float()
denom_even = torch.pow(10000, even_i/d_model)

In [4]:
odd_i = torch.arange(1,d_model, 2).float()
denom_odd = torch.pow(10000, (odd_i-1)/d_model)

In [5]:
denom_odd, denom_even

(tensor([  1.0000,  21.5443, 464.1590]),
 tensor([  1.0000,  21.5443, 464.1590]))

In [6]:
position = torch.arange(0, max_seq_len).reshape(max_seq_len,1)
position

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])

In [7]:
even_pe = torch.sin(position/denom_even)
odd_pe = torch.cos(position / denom_odd)

In [8]:
even_pe,odd_pe

(tensor([[ 0.0000,  0.0000,  0.0000],
         [ 0.8415,  0.0464,  0.0022],
         [ 0.9093,  0.0927,  0.0043],
         [ 0.1411,  0.1388,  0.0065],
         [-0.7568,  0.1846,  0.0086],
         [-0.9589,  0.2300,  0.0108],
         [-0.2794,  0.2749,  0.0129],
         [ 0.6570,  0.3192,  0.0151],
         [ 0.9894,  0.3629,  0.0172],
         [ 0.4121,  0.4057,  0.0194]]),
 tensor([[ 1.0000,  1.0000,  1.0000],
         [ 0.5403,  0.9989,  1.0000],
         [-0.4161,  0.9957,  1.0000],
         [-0.9900,  0.9903,  1.0000],
         [-0.6536,  0.9828,  1.0000],
         [ 0.2837,  0.9732,  0.9999],
         [ 0.9602,  0.9615,  0.9999],
         [ 0.7539,  0.9477,  0.9999],
         [-0.1455,  0.9318,  0.9999],
         [-0.9111,  0.9140,  0.9998]]))

In [10]:
x= torch.zeros((10,6))
x[:,0::2]= even_pe

In [12]:
x[:,1::2]=odd_pe

In [13]:
x

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000],
        [ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000],
        [ 0.1411, -0.9900,  0.1388,  0.9903,  0.0065,  1.0000],
        [-0.7568, -0.6536,  0.1846,  0.9828,  0.0086,  1.0000],
        [-0.9589,  0.2837,  0.2300,  0.9732,  0.0108,  0.9999],
        [-0.2794,  0.9602,  0.2749,  0.9615,  0.0129,  0.9999],
        [ 0.6570,  0.7539,  0.3192,  0.9477,  0.0151,  0.9999],
        [ 0.9894, -0.1455,  0.3629,  0.9318,  0.0172,  0.9999],
        [ 0.4121, -0.9111,  0.4057,  0.9140,  0.0194,  0.9998]])

In [33]:
import torch
import torch.nn

class PositionalEncoding(nn.Module):
    def __init__(self,max_seq_len:int, d_model:int):
        super().__init__()
        self.maxseq = max_seq_len
        self.d_model = d_model
        
    def forward(self):
        pos = torch.arange(0, self.maxseq).reshape(self.maxseq,1)
        i = torch.arange(0, d_model, 2)
        denom = torch.pow(10000, i/self.d_model)
        odd_pos = torch.cos(pos/denom)
        even_pos = torch.sin(pos/denom)
        x_das = torch.zeros(self.maxseq,self.d_model)
        x_das[:,0::2] = even_pos
        x_das[:,1::2] = odd_pos
        return x_das
        
        

In [38]:
model = PositionalEncoding(10,6)
a= model.forward()
a.shape

torch.Size([10, 6])

In [42]:
a

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0464,  0.9989,  0.0022,  1.0000],
        [ 0.9093, -0.4161,  0.0927,  0.9957,  0.0043,  1.0000],
        [ 0.1411, -0.9900,  0.1388,  0.9903,  0.0065,  1.0000],
        [-0.7568, -0.6536,  0.1846,  0.9828,  0.0086,  1.0000],
        [-0.9589,  0.2837,  0.2300,  0.9732,  0.0108,  0.9999],
        [-0.2794,  0.9602,  0.2749,  0.9615,  0.0129,  0.9999],
        [ 0.6570,  0.7539,  0.3192,  0.9477,  0.0151,  0.9999],
        [ 0.9894, -0.1455,  0.3629,  0.9318,  0.0172,  0.9999],
        [ 0.4121, -0.9111,  0.4057,  0.9140,  0.0194,  0.9998]])